<a href="https://colab.research.google.com/github/ashu-94/POWER_BI_DASHBOARD_SQL_PROJECTS_OCTOBER/blob/main/Rag_Application_Using_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Objective**
  * **We are going to create our conversational AI, that will answer the questions based on the given data source (pdf, text, img)**

* **`Open Source Model`: Deepseek, Mixtral, Zephyr, Dolly, Llama, Phi (HuggingFace, Unsloth, replicate)**

* **`Proprietry Models`: OpenAI, Google Gemini & PaLm, Microsoft**

* **`ollama`: Model Provider, that helps you to run your model locally on your device**

### **RAG Application**
* **Indexing**
  * **Load the data: Document Loader**
  * **Split the data: Text Splitter**
  * **Embed the data: Embedding Model**
  * **Save the data into a DB: VectorDB (`Chroma` and PineCone)**
<hr>
* **Retrieval**
  * **Setup LLM: ChatGPT (4o-mini, GPT-4)**
  * **Prompt Engineering (To make sure the model works fine)**
  * **Connect & Chain these all together: Chain**
  * **Utilize the LLM: Test**
<hr>
  * **Interface for having results as output: Gradio**

# **Step 1 - Requirement Phase**

* **Data Source: `plain text file`**
* **Framework: `Langchain`**

In [ ]:
!pip install langchain langchain_community langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.7 MB/s eta 0:00:00


### **Importing dependencies**

In [ ]:
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser

## **Step 2 Document Processing**

### **1. Taking Plain Text File**

In [ ]:
with open("/content/animal_kingdom_overview.txt")as f:
  files=f.read() #this method is used to read text file only.

In [ ]:
files # all these data of text file is loaded in file...

'Animal Kingdom Overview\n=======================\n\n1. Introduction\n---------------\nThe Animal Kingdom, also known as Kingdom Animalia, encompasses all multicellular, eukaryotic organisms that are heterotrophic in nature. These organisms are capable of locomotion at some stage of their life cycle and primarily reproduce sexually. Animals are characterized by their ability to respond quickly to stimuli and their complex tissue structures.\n\n2. Major Phyla\n--------------\n- **Porifera**: Simplest animals, commonly known as sponges.\n- **Cnidaria**: Aquatic animals with stinging cells, such as jellyfish and corals.\n- **Platyhelminthes**: Flatworms with bilateral symmetry and no body cavity.\n- **Nematoda**: Roundworms with a pseudocoelom and complete digestive system.\n- **Annelida**: Segmented worms like earthworms and leeches.\n- **Arthropoda**: Largest phylum including insects, arachnids, and crustaceans.\n- **Mollusca**: Soft-bodied animals, often with a shell, like snails and o

### **2. Split the data**

* **Context window**

  * **GPT4: 8192 Token (input and output)**

  * **(input) 4000 words + 4192 words (output)**

In [ ]:
TextSplitter=CharacterTextSplitter(#split the data with no of characters
    chunk_size=1000, # all chunks of 1000 characters
    chunk_overlap=200, # to maintain continuity
    length_function=len
    )

### **3. Create the split / segment the documentation**

In [ ]:
texts=TextSplitter.create_documents([files])

In [ ]:
texts

[Document(metadata={}, page_content='Animal Kingdom Overview\n=======================\n\n1. Introduction\n---------------\nThe Animal Kingdom, also known as Kingdom Animalia, encompasses all multicellular, eukaryotic organisms that are heterotrophic in nature. These organisms are capable of locomotion at some stage of their life cycle and primarily reproduce sexually. Animals are characterized by their ability to respond quickly to stimuli and their complex tissue structures.'),
 Document(metadata={}, page_content='2. Major Phyla\n--------------\n- **Porifera**: Simplest animals, commonly known as sponges.\n- **Cnidaria**: Aquatic animals with stinging cells, such as jellyfish and corals.\n- **Platyhelminthes**: Flatworms with bilateral symmetry and no body cavity.\n- **Nematoda**: Roundworms with a pseudocoelom and complete digestive system.\n- **Annelida**: Segmented worms like earthworms and leeches.\n- **Arthropoda**: Largest phylum including insects, arachnids, and crustaceans.\n-

### **Output**

In [ ]:
len(texts)# 4chunks

4

In [ ]:
texts[0]

Document(metadata={}, page_content='Animal Kingdom Overview\n=======================\n\n1. Introduction\n---------------\nThe Animal Kingdom, also known as Kingdom Animalia, encompasses all multicellular, eukaryotic organisms that are heterotrophic in nature. These organisms are capable of locomotion at some stage of their life cycle and primarily reproduce sexually. Animals are characterized by their ability to respond quickly to stimuli and their complex tissue structures.')

In [ ]:
texts[3]

Document(metadata={}, page_content='5. Conclusion\n-------------\nAnimals play a crucial role in maintaining ecological balance. They contribute to food chains, pollination, decomposition, and serve as companions and sources of food and labor for humans. Understanding the diversity of the Animal Kingdom helps us appreciate the complexity and interdependence of life on Earth.')

# **Step 3 - Embed the data using Embedding Model**

### **Create the embeddings**

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embedding_model=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-V2')

/tmp/ipython-input-2537108836.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-V2')
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **Database Formation**

In [ ]:
vector_db=Chroma(
    collection_name='animal_kingdom_overview',#Database Name
    embedding_function=embedding_model,
)

In [ ]:
vector_db

### **Load the documents in the DB**

In [ ]:
storage_id=vector_db.add_documents(texts)

In [ ]:
len(storage_id)

4

In [ ]:
storage_id[1]

'73a18db8-5a8c-410b-be9f-07d830cf3e7e'

In [ ]:
storage_id[2]

'3dfb0ed4-5209-4974-9975-a3e85335f06d'

In [ ]:
len(storage_id) == len(texts)# length of storage id and length of texts will be same.

True

1. Text

   └── Raw input text data (e.g., document, web page, transcript)

2. Split into Chunks

   └── Divide text into manageable chunks (e.g., by sentences or paragraphs)

3. Embedding Model

   └── Use a model (like OpenAI, Sentence-BERT) to convert text chunks into embeddings

4. Vectors

   └── Embeddings are high-dimensional numeric representations of the text

5. Vector Database

   └── Store these vectors in a database optimized for similarity search (e.g., FAISS, Pinecone, Weaviate)

6. Primary IDs

   └── Assign a unique identifier to each vector entry

7. Ensure Uniqueness

   └── Validate that each ID is distinct to avoid collisions or duplication


### **Similarity Searching using VecDB**

In [ ]:
#res=vector_db.similarity_search(
 #   query='What is the animal kingdom?',
 #   k=2 # top 2 matching vector/result using simalarity search
#)


In [ ]:
#res[0]

Document(id='20a91fd0-f64e-489e-a8f1-96fec5954397', metadata={}, page_content='Animal Kingdom Overview\n=======================\n\n1. Introduction\n---------------\nThe Animal Kingdom, also known as Kingdom Animalia, encompasses all multicellular, eukaryotic organisms that are heterotrophic in nature. These organisms are capable of locomotion at some stage of their life cycle and primarily reproduce sexually. Animals are characterized by their ability to respond quickly to stimuli and their complex tissue structures.')

In [ ]:
#res[1]

Document(id='c2b772c8-6c19-408f-a653-ba6aed80ebb9', metadata={}, page_content='5. Conclusion\n-------------\nAnimals play a crucial role in maintaining ecological balance. They contribute to food chains, pollination, decomposition, and serve as companions and sources of food and labor for humans. Understanding the diversity of the Animal Kingdom helps us appreciate the complexity and interdependence of life on Earth.')

# **Step 4 - Setting up the Retrievals**

## **Retrievals do the similarity search,similar vector (to fetch similar vectors) automatically**

### **a. Create a retriever**

In [ ]:
retriever=vector_db.as_retriever()

In [ ]:
retriever=vector_db.as_retriever(search_kwargs={"k":3})

#search for 3 similar context

### **b. LLM Instance**

In [ ]:
# llm= google/flan-t5 ---> base large small

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-large')

## **Padding in Tokenization**

In [ ]:
if tokenizer.pad_token is None:#if no tokenization is there then add
  tokenizer.add_special_tokens({'pad_token':'[Pad]'})

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-large')

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
model.resize_token_embeddings(len(tokenizer))#adding auto padding in models as well

Embedding(32100, 1024)

In [ ]:
model.config.pad_token_id=tokenizer.pad_token_id

# **Initialize Pipeline**

In [ ]:
generator=pipeline(
    'text2text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150
)

Device set to use cpu


# **HuggingFace Pipeline**

In [ ]:
from langchain_community.llms import HuggingFacePipeline

In [ ]:
llm=HuggingFacePipeline(pipeline=generator).to_device(model)

/tmp/ipython-input-3653542328.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=generator)


### **c. Design a Prompt**

In [ ]:
template = """
Use the context provided to answer the question at the end. If you don't know the answer,
just say that you don't know, don't try to make up an answer.
Answer only from context.

context: {context}

Question: {question}

Answer:
"""

In [ ]:
custom_temp=PromptTemplate(
    template=template,

)

In [ ]:
custom_temp

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nUse the context provided to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\ncontext: {context}\n\nQuestion: {question}\n\nAnswer:\n")

**We have a template, model, database**

* **Can we connect them**

In [ ]:
rag_chain=(
    {"context":retriever,"question":RunnablePassthrough()}
    |custom_temp
    |llm
    |StrOutputParser()
)

In [ ]:
# Input Question
#       |
# Retriever ->Format->Context
#       |
# {context,question}
#       |
# Prompt Template(custom_template)
#       |
#LLM (generate output)
#        |
#StrOutputParser(final output string human readable format )


# **Step 5 - Test**

In [ ]:
query="What is the animal kingdom?"
res=rag_chain.invoke(query)
print(res)

Token indices sequence length is longer than the specified maximum sequence length for this model (791 > 512). Running this sequence through the model will result in indexing errors


encompasses all multicellular, eukaryotic organisms that are heterotrophic in nature


In [ ]:
# get_ipython().system('pip install --force-reinstall requests==2.32.4')